# Iowa City Formatting

To do list here.

In [1]:
# Libraries

from postal.expand import expand_address
import pandas as pd
import pickle

In [2]:
# Directories

dataDir = '../../data/external/iowa-city-police/'
arrestsRaw = dataDir + 'arrest-loc-inc.csv'
citationsRaw = dataDir + 'citation.csv'

processDir = '../../data/processed/iowa-city-police/'

### Citations

In [3]:
citeCols = ['lName', 'fName', 'homeAddr', 'homeCity', 'dob',\
            'timestamp', 'charge', 'incAddr', 'incDate']

# read in: add cols, parse dates, ...
citeDF = pd.read_csv((citationsRaw), header=None, names=citeCols, \
                    parse_dates=['dob', 'incDate'])

# drop any rows that dont have a name or address associated with them
citeDF.dropna(subset=['lName', 'incAddr'], inplace=True)

In [4]:
citeDF.head(2)

,lName,fName,homeAddr,homeCity,dob,timestamp,charge,incAddr,incDate
0,ANDINO-PEREGRINO,JUDITH,509 EAST 1ST STREET,WEST LIBERTY,10/18/1955,NaN,NaN,WB HWY 6 / BOYRUM,2012-03-08
1,MCKINNEY,ROOSEVELT,3610 TAFT AVE SE,IOWA CITY,12/19/1940,NaN,NaN,1500 BLK ROCHESTER AVE,2012-03-16


In [5]:
# Variables

def formatAge(dob, inc):
    '''Function to get the timedelta from two dates in years (age)
    Input: two pandas parsed dates
    OutPut: Integer that represents num years'''
    
    if (dob or inc) in [None, False, ]
    
# needs normalized date cols
#citeDF['age'] = citeDF['incDate'] - citeDF['dob']

SyntaxError: invalid syntax (<ipython-input-5-18b1ff4605ca>, line 8)

In [6]:
# addresses

def formatAddr(raw):
    '''Function to format an address only if the raw string is valid.
    Input: String containing address
    Output: Normalized address'''
    
    # Error checking
    if ((type(raw) != str) | (len(raw) == 0)):
        return False
    
    # Prepare and get info
    query = raw + ', Iowa City, Iowa'
    expanded = expand_address(query)
    
    # return normalized address or notify
    directions = ['nb', 'eb', 'sb', 'wb']
    if (len(expanded) > 0):
        bestCand = expanded[0]
        if (bestCand[:2]) in directions:
            return bestCand[3:]
        else:
            return bestCand
    else:
        print (raw + ' did not expand!')
        
citeDF['incFormAddr'] = citeDF['incAddr'].apply(lambda x: formatAddr(x))

In [7]:
# do the charges
# cluster them

In [8]:
# Save out
citeDF.to_csv((processDir + 'icpd-cites.csv'), index=False)
citeDF.to_pickle((processDir + 'icpd-cites.pickle'))

-------------------

### Arrests

In [9]:
arrCols = ['name', 'dob', 'homeAddr', 'homeCity', 'timestamp',\
          'incDate', 'incAddr', 'incAct', 'charge']

# read in: add cols, parse dates, ...
arrDF = pd.read_csv((arrestsRaw), header=None, names=arrCols,\
                    parse_dates=['dob', 'incDate'])
print ('{} arrests'.format(arrDF.size))

# drop any rows that dont have a name or address associated with them
arrDF.dropna(subset=['name', 'incAddr'], inplace=True)

532134 arrests


In [10]:
arrDF['incFormAddr'] = arrDF['incAddr'].apply(lambda x: formatAddr(x))

In [11]:
arrDF.head(3)

,name,dob,homeAddr,homeCity,timestamp,incDate,incAddr,incAct,charge,incFormAddr
0,"Hamann, Brian Joel",1980-09-18,97 Taupe Ln,Reeds Spring,2.006014e+09,2006-03-26,1 Av S/Court St,Taken into Custody,Public Intoxication,1 avenue south court street iowa city iowa
1,"PATHEUANGSIN, WALLY",1991-03-17,1903 Hollywood Blvd,Iowa City,2.007032e+09,2007-06-27,1 Av S/Court St,Taken into Custody,Operate Veh Without Owners Consent,1 avenue south court street iowa city iowa
2,"HOWARD, CURLEY LEE",2066-02-01,2401 Hwy 6 E 3007,Iowa City,2.010004e+09,2010-01-24,1 Av S/Lower Muscatine Rd,On View,DRIVING WHILE BARRED,1 avenue south lower muscatine road iowa city...


In [ ]:
# timestamps
def formatStamps(raw):
    year = raw[:5]
    seconds = raw[5:]

In [ ]:
# save out
arrDF.to_csv((processDir + 'arrests-icpd.csv'), index=False)
arrDF.to_pickle((processDir + 'arrests-icpd.pickle'))

---

## **Analysis**

### Charges

What are the most popular charges? How can we normalize the charges into tidy bins?

In [12]:
# lower is first preprocessing step
citeDF['charge'] = citeDF.charge.str.lower()
arrDF.charge = arrDF.charge.str.lower()

In [13]:
# value counts
charges = pd.concat([citeDF['charge'], arrDF['charge']], ignore_index=True)
chargeCounts = pd.DataFrame({'num': charges.value_counts(), 'perc':charges.value_counts(normalize=True)})

print ('There are {} charges (not fixed for errors)'.format(chargeCounts.size))

chargeCounts.reset_index(inplace=True, drop=False)
chargeCounts.rename(columns={'index': 'charge'}, inplace=True)

There are 1064 charges (not fixed for errors)


In [16]:
chargeCounts.head().style.bar(subset=['num', 'perc'])

In [17]:
# save that ish
chargeCounts.to_csv((processDir + 'charge-counts-icpd.csv'), index=False)
chargeCounts.to_pickle((processDir + 'charge-counts-icpd.pickle'))

In [18]:
chargeGroups = pd.read_csv((processDir + 'charge-groups-icpd.csv'))
chargeGroups.head()

,charge,subcat,num,perc
0,speeding,speeding,13918,0.128081
1,public intoxication,public intoxication,10633,0.097850
2,paula,paula,6290,0.057884
3,no proof of insurance,insurance,4562,0.041982
4,no seatbelt/improper seatbelt,seatbelt,4230,0.038927


In [30]:
chargeGroups[['subcat', 'num']].groupby('subcat').agg('sum')\
    .sort_values('num', ascending=False)\
    .style.bar(subset=['num'])

### Addresses